# 🔍 Uso del Modelo Entrenado - Detección de Fraude

Este notebook permite cargar y usar el modelo XGBoost entrenado para predecir fraudes en transacciones de tarjetas de crédito.

## Instrucciones:
1. Asegúrate de tener los archivos del modelo en la carpeta `Modelo_Entrenado/`
2. Ejecuta las celdas en orden
3. Puedes usar el modelo con datos individuales o en lote

In [8]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import joblib
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("Librerías importadas correctamente")

Librerías importadas correctamente


## 1. Cargar Modelo y Artefactos

In [9]:
# Cargar el modelo entrenado
modelo_path = 'Modelo_Entrenado/modelo_xgboost_fraude.pkl'
scaler_path = 'Modelo_Entrenado/scaler_robust.pkl'
features_path = 'Modelo_Entrenado/features_names.pkl'

# Verificar que los archivos existan
if not os.path.exists(modelo_path):
    raise FileNotFoundError(f"No se encontró el modelo en: {modelo_path}")
if not os.path.exists(scaler_path):
    raise FileNotFoundError(f"No se encontró el scaler en: {scaler_path}")
if not os.path.exists(features_path):
    raise FileNotFoundError(f"No se encontraron los nombres de features en: {features_path}")

# Cargar artefactos
print("Cargando modelo y artefactos...")
modelo = joblib.load(modelo_path)
scaler = joblib.load(scaler_path)
feature_names = joblib.load(features_path)

print(f"[OK] Modelo cargado exitosamente")
print(f"[OK] Scaler cargado exitosamente")
print(f"[OK] Features esperados: {len(feature_names)}")
print(f"\nFeatures requeridos: {', '.join(feature_names)}")

Cargando modelo y artefactos...
[OK] Modelo cargado exitosamente
[OK] Scaler cargado exitosamente
[OK] Features esperados: 29

Features requeridos: V1, V2, V3, V4, V5, V6, V7, V8, V9, V10, V11, V12, V13, V14, V15, V16, V17, V18, V19, V20, V21, V22, V23, V24, V25, V26, V27, V28, Amount


## 2. Función para Hacer Predicciones

In [10]:
def predecir_fraude(datos, mostrar_detalles=True):
    """
    Función para predecir si una transacción es fraude o no.
    
    Parámetros:
    -----------
    datos : pandas.DataFrame o dict
        Datos de transacción(es). Debe contener todas las características necesarias.
        Si es un dict, se convierte a DataFrame de una sola fila.
    mostrar_detalles : bool
        Si True, muestra información detallada de las predicciones.
    
    Returns:
    --------
    resultados : pandas.DataFrame
        DataFrame con predicciones y probabilidades
    """
    
    # Convertir dict a DataFrame si es necesario
    if isinstance(datos, dict):
        datos = pd.DataFrame([datos])
    elif not isinstance(datos, pd.DataFrame):
        raise ValueError("Los datos deben ser un pandas.DataFrame o un dict")
    
    # Verificar que tenga todas las columnas necesarias
    datos_original = datos.copy()
    columnas_faltantes = set(feature_names) - set(datos.columns)
    if columnas_faltantes:
        raise ValueError(f"Faltan las siguientes columnas: {columnas_faltantes}")
    
    # Seleccionar solo las columnas necesarias y en el orden correcto
    datos_features = datos[feature_names].copy()
    
    # Aplicar escalado
    datos_scaled = scaler.transform(datos_features)
    datos_scaled_df = pd.DataFrame(datos_scaled, columns=feature_names)
    
    # Hacer predicciones
    predicciones = modelo.predict(datos_scaled_df)
    probabilidades = modelo.predict_proba(datos_scaled_df)[:, 1]  # Probabilidad de fraude
    
    # Crear DataFrame con resultados
    resultados = pd.DataFrame({
        'Prediccion': predicciones,
        'Clase_Predicha': ['Fraude' if p == 1 else 'Normal' for p in predicciones],
        'Probabilidad_Fraude': probabilidades,
        'Probabilidad_Normal': 1 - probabilidades,
        'Confianza': [max(p, 1-p) for p in probabilidades]
    })
    
    # Combinar con datos originales si hay columnas adicionales
    if len(datos_original.columns) > len(feature_names):
        columnas_extra = [col for col in datos_original.columns if col not in feature_names]
        resultados = pd.concat([datos_original[columnas_extra].reset_index(drop=True), 
                               resultados], axis=1)
    
    # Mostrar resultados si se solicita
    if mostrar_detalles:
        print("="*80)
        print("RESULTADOS DE PREDICCIÓN")
        print("="*80)
        for idx, row in resultados.iterrows():
            print(f"\nTransacción #{idx + 1}:")
            print(f"  Predicción: {row['Clase_Predicha']}")
            print(f"  Probabilidad de Fraude: {row['Probabilidad_Fraude']:.4f} ({row['Probabilidad_Fraude']*100:.2f}%)")
            print(f"  Probabilidad de Normal: {row['Probabilidad_Normal']:.4f} ({row['Probabilidad_Normal']*100:.2f}%)")
            print(f"  Confianza: {row['Confianza']:.4f} ({row['Confianza']*100:.2f}%)")
            
            # Recomendación
            if row['Prediccion'] == 1:
                print(f"  ⚠️  RECOMENDACIÓN: BLOQUEAR TRANSACCIÓN - Alto riesgo de fraude")
            elif row['Probabilidad_Fraude'] > 0.3:
                print(f"  ⚠️  RECOMENDACIÓN: REVISAR MANUALMENTE - Probabilidad moderada de fraude")
            else:
                print(f"  ✅ RECOMENDACIÓN: APROBAR - Bajo riesgo de fraude")
        print("\n" + "="*80)
    
    return resultados

print("[OK] Función de predicción definida")

[OK] Función de predicción definida


## 3. Ejemplo: Predicción de una Transacción Individual

In [11]:
# Ejemplo de una transacción individual (como diccionario)
# NOTA: Estos son valores de ejemplo. Reemplaza con datos reales.

transaccion_ejemplo = {
    'V1': -1.359807134,
    'V2': -0.072781173,
    'V3': 2.536346738,
    'V4': 1.378155224,
    'V5': -0.33826177,
    'V6': 0.46238804,
    'V7': 0.239598554,
    'V8': 0.098697901,
    'V9': 0.36378697,
    'V10': 0.090794172,
    'V11': -0.55159953,
    'V12': -0.617800856,
    'V13': -0.991389847,
    'V14': -2.261873095,
    'V15': 0.524979725,
    'V16': 0.247998153,
    'V17': 0.771679402,
    'V18': 0.909412262,
    'V19': -0.68928096,
    'V20': -0.327641834,
    'V21': -0.139096572,
    'V22': -0.055352794,
    'V23': -0.059751841,
    'V24': 0.342207708,
    'V25': 0.389796345,
    'V26': 0.005857858,
    'V27': -0.013406374,
    'V28': -0.017969444,
    'Amount': 149.62
}

# Hacer predicción
resultado = predecir_fraude(transaccion_ejemplo)

# Mostrar resultado en formato tabla
print("\nResultado en formato DataFrame:")
print(resultado[['Clase_Predicha', 'Probabilidad_Fraude', 'Probabilidad_Normal', 'Confianza']])

RESULTADOS DE PREDICCIÓN

Transacción #1:
  Predicción: Normal
  Probabilidad de Fraude: 0.3729 (37.29%)
  Probabilidad de Normal: 0.6271 (62.71%)
  Confianza: 0.6271 (62.71%)
  ⚠️  RECOMENDACIÓN: REVISAR MANUALMENTE - Probabilidad moderada de fraude


Resultado en formato DataFrame:
  Clase_Predicha  Probabilidad_Fraude  Probabilidad_Normal  Confianza
0         Normal              0.37295              0.62705    0.62705


## 4. Ejemplo: Predicción en Lote (Múltiples Transacciones)

In [12]:
# EJEMPLO: Crear datos de ejemplo (simulados)
print("Creando ejemplo con múltiples transacciones...")

# Generar algunas transacciones de ejemplo (valores aleatorios dentro de rangos típicos)
np.random.seed(42)
num_transacciones = 5

datos_lote_ejemplo = pd.DataFrame({
    'V1': np.random.uniform(-3, 3, num_transacciones),
    'V2': np.random.uniform(-3, 3, num_transacciones),
    'V3': np.random.uniform(-3, 3, num_transacciones),
    'V4': np.random.uniform(-3, 3, num_transacciones),
    'V5': np.random.uniform(-3, 3, num_transacciones),
    'V6': np.random.uniform(-3, 3, num_transacciones),
    'V7': np.random.uniform(-3, 3, num_transacciones),
    'V8': np.random.uniform(-3, 3, num_transacciones),
    'V9': np.random.uniform(-3, 3, num_transacciones),
    'V10': np.random.uniform(-3, 3, num_transacciones),
    'V11': np.random.uniform(-3, 3, num_transacciones),
    'V12': np.random.uniform(-3, 3, num_transacciones),
    'V13': np.random.uniform(-3, 3, num_transacciones),
    'V14': np.random.uniform(-3, 3, num_transacciones),
    'V15': np.random.uniform(-3, 3, num_transacciones),
    'V16': np.random.uniform(-3, 3, num_transacciones),
    'V17': np.random.uniform(-3, 3, num_transacciones),
    'V18': np.random.uniform(-3, 3, num_transacciones),
    'V19': np.random.uniform(-3, 3, num_transacciones),
    'V20': np.random.uniform(-3, 3, num_transacciones),
    'V21': np.random.uniform(-3, 3, num_transacciones),
    'V22': np.random.uniform(-3, 3, num_transacciones),
    'V23': np.random.uniform(-3, 3, num_transacciones),
    'V24': np.random.uniform(-3, 3, num_transacciones),
    'V25': np.random.uniform(-3, 3, num_transacciones),
    'V26': np.random.uniform(-3, 3, num_transacciones),
    'V27': np.random.uniform(-3, 3, num_transacciones),
    'V28': np.random.uniform(-3, 3, num_transacciones),
    'Amount': np.random.uniform(50, 24000, num_transacciones)
})

print(f"Transacciones de ejemplo creadas: {len(datos_lote_ejemplo)}")

# Hacer predicciones en lote
resultados_lote = predecir_fraude(datos_lote_ejemplo, mostrar_detalles=True)

# Mostrar resumen
print("\n" + "="*80)
print("RESUMEN DE PREDICCIONES")
print("="*80)
print(f"Total de transacciones analizadas: {len(resultados_lote)}")
print(f"Predicciones de Fraude: {(resultados_lote['Prediccion'] == 1).sum()}")
print(f"Predicciones de Normal: {(resultados_lote['Prediccion'] == 0).sum()}")
print(f"\nProbabilidad promedio de fraude: {resultados_lote['Probabilidad_Fraude'].mean():.4f}")
print(f"Probabilidad máxima de fraude: {resultados_lote['Probabilidad_Fraude'].max():.4f}")
print(f"Probabilidad mínima de fraude: {resultados_lote['Probabilidad_Fraude'].min():.4f}")

Creando ejemplo con múltiples transacciones...
Transacciones de ejemplo creadas: 5
RESULTADOS DE PREDICCIÓN

Transacción #1:
  Predicción: Normal
  Probabilidad de Fraude: 0.0000 (0.00%)
  Probabilidad de Normal: 1.0000 (100.00%)
  Confianza: 1.0000 (100.00%)
  ✅ RECOMENDACIÓN: APROBAR - Bajo riesgo de fraude

Transacción #2:
  Predicción: Normal
  Probabilidad de Fraude: 0.0008 (0.08%)
  Probabilidad de Normal: 0.9992 (99.92%)
  Confianza: 0.9992 (99.92%)
  ✅ RECOMENDACIÓN: APROBAR - Bajo riesgo de fraude

Transacción #3:
  Predicción: Normal
  Probabilidad de Fraude: 0.0116 (1.16%)
  Probabilidad de Normal: 0.9884 (98.84%)
  Confianza: 0.9884 (98.84%)
  ✅ RECOMENDACIÓN: APROBAR - Bajo riesgo de fraude

Transacción #4:
  Predicción: Normal
  Probabilidad de Fraude: 0.1719 (17.19%)
  Probabilidad de Normal: 0.8281 (82.81%)
  Confianza: 0.8281 (82.81%)
  ✅ RECOMENDACIÓN: APROBAR - Bajo riesgo de fraude

Transacción #5:
  Predicción: Normal
  Probabilidad de Fraude: 0.0132 (1.32%)
  Prob

## 5. Uso con Archivo CSV (Descomentar para usar)

In [13]:
# INSTRUCCIONES:
# 1. Coloca tu archivo CSV con las transacciones en el mismo directorio
# 2. Asegúrate de que el CSV tenga todas las columnas: V1, V2, ..., V28, Amount
# 3. Descomenta y ejecuta el código siguiente
# 4. Los resultados se guardarán en 'predicciones_resultados.csv'

# Ruta del archivo CSV con las transacciones
# archivo_csv = 'transacciones_nuevas.csv'  # Cambia por tu archivo
#
# # Cargar datos
# try:
#     datos_csv = pd.read_csv(archivo_csv)
#     print(f"[OK] Archivo cargado: {len(datos_csv)} transacciones")
#     print(f"Columnas en el archivo: {datos_csv.columns.tolist()}")
#     
#     # Si el CSV tiene una columna 'id' u otra columna adicional, se conservará
#     # Hacer predicciones
#     resultados_csv = predecir_fraude(datos_csv, mostrar_detalles=False)
#     
#     # Guardar resultados
#     archivo_resultados = 'predicciones_resultados.csv'
#     resultados_csv.to_csv(archivo_resultados, index=False)
#     print(f"\n[OK] Resultados guardados en: {archivo_resultados}")
#     print(f"\nResumen:")
#     print(f"  - Total: {len(resultados_csv)}")
#     print(f"  - Fraudes detectados: {(resultados_csv['Prediccion'] == 1).sum()}")
#     print(f"  - Transacciones normales: {(resultados_csv['Prediccion'] == 0).sum()}")
#     
# except FileNotFoundError:
#     print(f"Error: No se encontró el archivo '{archivo_csv}'")
#     print("Por favor, verifica la ruta del archivo")
# except Exception as e:
#     print(f"Error al procesar el archivo: {str(e)}")

print("Celda lista para usar. Descomenta el código para cargar tu archivo CSV.")

Celda lista para usar. Descomenta el código para cargar tu archivo CSV.


## 6. Información del Modelo

In [14]:
# Mostrar información del modelo cargado
print("="*80)
print("INFORMACIÓN DEL MODELO")
print("="*80)
print(f"\nTipo de modelo: {type(modelo).__name__}")
print(f"Número de características: {len(feature_names)}")
print(f"\nCaracterísticas esperadas:")
for i, feat in enumerate(feature_names, 1):
    print(f"  {i:2d}. {feat}")

# Intentar cargar metadatos si existen
metadata_path = 'Modelo_Entrenado/metadata_modelo.txt'
if os.path.exists(metadata_path):
    print("\n" + "="*80)
    print("METADATOS DEL MODELO")
    print("="*80)
    with open(metadata_path, 'r', encoding='utf-8') as f:
        print(f.read())
else:
    print("\n⚠️  Archivo de metadatos no encontrado")

INFORMACIÓN DEL MODELO

Tipo de modelo: XGBClassifier
Número de características: 29

Características esperadas:
   1. V1
   2. V2
   3. V3
   4. V4
   5. V5
   6. V6
   7. V7
   8. V8
   9. V9
  10. V10
  11. V11
  12. V12
  13. V13
  14. V14
  15. V15
  16. V16
  17. V17
  18. V18
  19. V19
  20. V20
  21. V21
  22. V22
  23. V23
  24. V24
  25. V25
  26. V26
  27. V27
  28. V28
  29. Amount

METADATOS DEL MODELO
METADATOS DEL MODELO DE DETECCIÓN DE FRAUDE

Fecha de entrenamiento: 2026-01-08 22:09:52
Tipo de modelo: XGBoost Classifier
Objetivo: binary:logistic
Métrica de evaluación: aucpr

Parámetros del modelo:
  - max_depth: 6
  - learning_rate: 0.1
  - n_estimators: 200
  - subsample: 0.8
  - colsample_bytree: 0.8

Rendimiento en conjunto de prueba:
  - Accuracy: 0.9994
  - Precision (Fraude): 0.9989
  - Recall (Sensibilidad): 1.0000
  - F1-Score: 0.9994
  - AUC-ROC: 1.0000
  - AUC-PR: 1.0000

Total de features: 29
Features: V1, V2, V3, V4, V5, V6, V7, V8, V9, V10, V11, V12, V13, V

## 📝 Notas Importantes

1. **Precisión del Modelo**: El modelo tiene las siguientes métricas:
   - Accuracy: 99.94%
   - Precision (Fraude): 99.89%
   - Recall (Sensibilidad): 100%
   - AUC-ROC: 99.998%

2. **Umbral de Decisión**: El modelo usa un umbral de 0.5 por defecto. Puedes ajustar esto si necesitas más sensibilidad o especificidad.

3. **Características Importantes**: Las características más importantes para la detección son:
   - V14 (38.8%)
   - V10 (25.0%)
   - V4 (8.2%)

4. **Recomendaciones de Uso**:
   - Para predicciones individuales, usa la función `predecir_fraude()` con un diccionario
   - Para predicciones en lote, usa la función con un DataFrame de pandas
   - Siempre valida los datos antes de hacer predicciones
   - Guarda los resultados para auditoría y análisis posterior

5. **Mantenimiento**: El modelo debe ser re-entrenado periódicamente con nuevos datos para mantener su efectividad.